In [7]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("hokarami/TEDAM3")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [9]:
q = runs_df['name'].str.contains('\[R2-')
runs_df = runs_df[q]
runs_df

,summary,config,name
120,"{'Valid-pred_label/AUROC': 0.9709441764300482,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1147879
121,"{'Test-pred_label/loss': 0, 'Valid-pred_label/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1147879
122,"{'Test-pred_label/loss': 0, 'Valid-pred_label/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1147879
123,"{'Train-pred_label/AUPRC': 0.9614611373558438,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1147879
124,{'Train-pred_label/f1-binary': 0.9142826586813...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1147879
140,"{'Valid-CIF/NLL': 1626.0362548828125, 'Train-N...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1146925
141,"{'Train-pred_label/AUPRC': 0.9790106725641492,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1146925
142,{'Valid-NextType(ML)/precision-weighted': 0.60...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1146925
143,{'Best-Test-pred_label/AUPRC': 0.8660379757164...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1146925
144,"{'Valid-CIF/NLL': -7888.9765625, 'Test-pred_la...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R2-mc1]TEDA__shpmarklabel-1146925


In [10]:

runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

dict_keys(['Valid-pred_label/AUROC', 'Valid-pred_label/f1-binary', 'Valid-NextType(ML)/auc-ovo-weighted', 'Test-pred_label/ACC', 'Valid-NextTime/RMSE', 'Test-CIF/NLL-#events', 'Train-NextTime/msae', 'Train-pred_label/AUROC', 'Best-Valid-pred_label/AUROC', 'Valid-pred_label/recall-binary', 'Test-NextType(ML)/recall-weighted', 'status', 'Valid-CIF/NLL', 'Train-NextTime/RMSE', 'Valid-pred_label/precision-binary', 'Train-pred_label/precision-binary', 'Test-NextTime/rmse_norm', 'Test-NextType(ML)/f1-weighted', 'Test-pred_label/recall-binary', 'Train-NextType(ML)/recall-weighted', 'Valid-CIF/#events', 'Test-NextTime/RMSE', 'Valid-pred_label/ACC', 'Train-pred_label/AUPRC', 'Test-NextType(ML)/precision-weighted', 'Valid-pred_label/AUPRC', 'Train-pred_label/f1-binary', 'Valid-NextType(ML)/f1-weighted', 'Test-pred_label/f1-binary', 'Train-NextType(ML)/f1-weighted', 'Valid-NextType(ML)/auc-PR-weighted', 'Valid-NextType(ML)/precision-weighted', 'max_obj_val', 'Train-CIF/NLL', 'Train-CIF/#events', 

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [14]:
i_by =['dataset','setting','test_center','event_enc']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
# i_metric = ['Test-CIF/NLL-#events']



runs_df.iloc[0].config.keys()

d_name = [x for x in runs_df.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in runs_df.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in runs_df.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = runs_df.name
df = df[i_by + ['test_center','split']+i_metric]
df




a0 = df.groupby(i_by)[i_metric].size().rename('runs')

a1 = df.groupby(i_by)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_by)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_by[:2],columns=i_by[-1])

a1

a1.to_csv("aaa.csv")

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

,dataset,setting,event_enc,test_center,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC
0,/scratch/hokarami/data_tedam/p19/,mc1,1,1,0,0.644970,0.801508
1,/scratch/hokarami/data_tedam/p19/,mc1,1,1,2,0.680739,0.886685
2,/scratch/hokarami/data_tedam/p19/,mc1,1,1,4,0.670487,0.846750
3,/scratch/hokarami/data_tedam/p19/,mc1,1,1,1,0.677966,0.760311
4,/scratch/hokarami/data_tedam/p19/,mc1,1,1,3,0.687719,0.848534
5,/scratch/hokarami/data_tedam/p19/,mc1,1,0,0,0.761658,0.896558
6,/scratch/hokarami/data_tedam/p19/,mc1,1,0,4,0.716667,0.886010
7,/scratch/hokarami/data_tedam/p19/,mc1,1,0,1,0.679245,0.890606
8,/scratch/hokarami/data_tedam/p19/,mc1,1,0,2,0.708502,0.866038
9,/scratch/hokarami/data_tedam/p19/,mc1,1,0,3,0.717241,0.865341


Best-Test-pred_label/f1-binary  \
dataset                           setting event_enc                                  
/scratch/hokarami/data_tedam/p19/ mc1     0                            0.76 (0.06)   
                                          1                            0.69 (0.03)   
                                  sc      0                            0.62 (0.05)   
                                          1                            0.63 (0.03)   

                                                    Best-Test-pred_label/AUPRC  
dataset                           setting event_enc                             
/scratch/hokarami/data_tedam/p19/ mc1     0                        0.92 (0.02)  
                                          1                        0.85 (0.04)  
                                  sc      0                        0.73 (0.05)  
                                          1                        0.72 (0.04)

Best-Test-pred_label/f1-binary  \
event_enc                                                              0   
dataset                           setting                                  
/scratch/hokarami/data_tedam/p19/ mc1                        0.76 (0.06)   
                                  sc                         0.62 (0.05)   

                                                        \
event_enc                                            1   
dataset                           setting                
/scratch/hokarami/data_tedam/p19/ mc1      0.69 (0.03)   
                                  sc       0.63 (0.03)   

                                          Best-Test-pred_label/AUPRC  \
event_enc                                                          0   
dataset                           setting                              
/scratch/hokarami/data_tedam/p19/ mc1                    0.92 (0.02)   
                                  sc                     0.73 (0.05)   

                                                        
event_enc                                            1  
dataset                           setting               
/scratch/hokarami/data_tedam/p19/ mc1      0.85 (0.04)  
                                  sc       0.72 (0.04)

Best-Test-pred_label/f1-binary  \
dataset                           setting event_enc                                  
/scratch/hokarami/data_tedam/p19/ mc1     0                            0.76 (0.06)   
                                          1                            0.69 (0.03)   
                                  sc      0                            0.62 (0.05)   
                                          1                            0.63 (0.03)   

                                                    Best-Test-pred_label/AUPRC  
dataset                           setting event_enc                             
/scratch/hokarami/data_tedam/p19/ mc1     0                        0.92 (0.02)  
                                          1                        0.85 (0.04)  
                                  sc      0                        0.73 (0.05)  
                                          1                        0.72 (0.04)

In [40]:
df3 = df2.reset_index()

df3.pivot(index=['dataset','setting'],columns=['event_enc'],values=['val','std'])

val                 \
event_enc                                              0              1   
dataset                           setting                                 
/scratch/hokarami/data_tedam/p12/ mc1      0.492 (0.028)  0.492 (0.026)   
                                  mc2      0.428 (0.005)  0.445 (0.007)   
                                  sc       0.405 (0.035)  0.425 (0.026)   
/scratch/hokarami/data_tedam/p19/ mc1      0.728 (0.061)  0.711 (0.038)   
                                  mc2      0.533 (0.095)   0.469 (0.05)   
                                  sc       0.619 (0.039)   0.632 (0.03)   

                                             std         
event_enc                                      0      1  
dataset                           setting                
/scratch/hokarami/data_tedam/p12/ mc1      0.028  0.026  
                                  mc2      0.005  0.007  
                                  sc       0.035  0.026  
/scratch/hokarami/data_tedam/p19/ mc1      0.061  0.038  
                                  mc2      0.095   0.05  
                                  sc       0.039   0.03

In [18]:
temp = df2.reset_index()[df2.reset_index().dataset=='/scratch/hokarami/data_tedam/p12/']
fig = px.histogram(temp, x='setting' ,y='mean',color='event_enc', barmode='group')
fig.show()